# NHL DATA SCRAPER v.2.0

### Step 1
Import all required modules

In [44]:
import os
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### Step 2
1. Global variables
2. Set up chrome options for automatic download

In [45]:
DOWNLOADS_PATH = 'C:\\Users\\riger\\Downloads'
SCRIPT_PATH = os.getcwd()
NHL_STATS_CSV_PATH = os.path.join(SCRIPT_PATH, './generated/nhl_stats_data.csv')
NHL_GOALIE_CSV_PATH = os.path.join(SCRIPT_PATH, './generated/nhl_goalie_data.csv')
COMBINED_CSV_PATH = os.path.join(SCRIPT_PATH, 'combined.csv')
BASE_STATS_URL = 'https://www.nhl.com/stats/teams?aggregate=0&report=daysbetweengames&reportType=game&seasonFrom=20212022&seasonTo=20242025&dateFromSeason&gameType=2&homeRoad=H&sort=a_gameDate&page={}&pageSize=100'
BASE_GOALIE_URL = 'https://www.nhl.com/stats/goalies?aggregate=0&reportType=game&seasonFrom=20212022&seasonTo=20242025&dateFromSeason&gameType=2&homeRoad=H&sort=a_gameDate&page={}&pageSize=100'

In [46]:
chrome_options = Options()
chrome_options.add_experimental_option('prefs', {
    'download.default_directory': DOWNLOADS_PATH,
    'download.prompt_for_download': False,
    'download.directory_upgrade': True,
    'safebrowsing.enabled': True
})

### Step 4
Start scraping stats
- This will open every page from 0 to 52
- It will download the csv to your downloads folder, read it, and save/concat it to the 'data' pandas df
- Finally, remove the downloaded file
- Repeat with all pages

In [47]:
#Driver setup

service = Service('chromedriver.exe')
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.maximize_window()

In [48]:
# try:
#     # Create csv directory if doesn't exist (will be used at the end)
#     if not os.path.exists(NHL_STATS_CSV_PATH):
#         os.makedirs('generated', exist_ok=True)  # create 'generated' folder if needed
        
#         with open(NHL_STATS_CSV_PATH, 'w') as f:
#             f.write("")
#             print("Created file:", NHL_STATS_CSV_PATH)

#     combined_data = pd.DataFrame()

#     for page in range(0, 53):
#         url = BASE_STATS_URL.format(page)
#         driver.get(url)

#         # Wait for the export link to be present and clickable
#         export_link = WebDriverWait(driver, 20).until(
#             EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/main/div/div/div/div[2]/div/div[2]/div/main/div[2]/h4/a'))
#         )

#         # Scroll to the element
#         driver.execute_script("arguments[0].scrollIntoView(true);", export_link)
#         export_link.click()
#         time.sleep(5)  # Wait for the file to download

#         # Process the downloaded file
#         downloaded_file = os.path.join(DOWNLOADS_PATH, 'Days between Games.xlsx')
#         if os.path.exists(downloaded_file):
#             data = pd.read_excel(downloaded_file)
#             combined_data = pd.concat([combined_data, data], ignore_index=True)
#             os.remove(downloaded_file)  # Remove the downloaded file after processing

#     # Save the combined data as a CSV file
#     if os.path.exists('Days between Games.xlsx'):
#         os.remove('Days between Games.xlsx')

#     combined_data.to_csv(NHL_STATS_CSV_PATH, index=False)
#     print(f"Combined CSV saved at: {NHL_STATS_CSV_PATH}")

# finally:
#     driver.quit()

### Step 5
Scrape goalie data
* This will scrape all pages from 0 to 56
* Like before, will download a csv, read into a pandas df
* combine all pages into one csv

In [ ]:
# try:
#     # Create csv directory if doesn't exist (will be used at the end)
#     if not os.path.exists(NHL_GOALIE_CSV_PATH):
#         if not os.path.exists('generated'):
#             os.makedirs('generated', exist_ok=True)  # create 'generated' folder if needed
        
#         with open(NHL_GOALIE_CSV_PATH, 'w') as f:
#             f.write("")
#             print("Created file:", NHL_GOALIE_CSV_PATH)

#     combined_data = pd.DataFrame()

#     for page in range(0, 56):
#         url = BASE_GOALIE_URL.format(page)
#         driver.get(url)

#         # Wait for the export link to be present and clickable
#         export_link = WebDriverWait(driver, 20).until(
#             EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/main/div/div/div/div[2]/div/div[2]/div/main/div[2]/h4/a'))
#         )

#         # Scroll to the element
#         driver.execute_script("arguments[0].scrollIntoView(true);", export_link)
#         export_link.click()
#         time.sleep(5)  # Wait for the file to download

#         # Process the downloaded file
#         downloaded_file = os.path.join(DOWNLOADS_PATH, 'Summary.xlsx')
#         if os.path.exists(downloaded_file):
#             data = pd.read_excel(downloaded_file)
#             combined_data = pd.concat([combined_data, data], ignore_index=True)
#             os.remove(downloaded_file)  # Remove the downloaded file after processing

#     # Save the combined data as a CSV file
#     if os.path.exists('Summary.xlsx'):
#         os.remove('Summary.xlsx')

#     combined_data.to_csv(NHL_GOALIE_CSV_PATH, index=False)
#     print(f"Combined CSV saved at: {NHL_GOALIE_CSV_PATH}")

# finally:
#     driver.quit()

Created file: c:\Users\riger\Desktop\projects\NHL_predictor\main\ml_dev\notebooks\./generated/nhl_goalie_data.csv
Combined CSV saved at: c:\Users\riger\Desktop\projects\NHL_predictor\main\ml_dev\notebooks\./generated/nhl_goalie_data.csv
